In [4]:
import cv2
import numpy as np

def find_marker(image):
    if image is None:
        raise ValueError("Image not loaded. Please check the file path.")
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(gray, 35, 125)

    # Find the contours of the edged image
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No contours found in the image. Check image quality or parameters.")
    c = max(contours, key=cv2.contourArea)

    return cv2.minAreaRect(c)

def distance_to_camera(knownWidth, focalLength, perWidth):
    # Compute and return the distance from the marker to the camera
    return (knownWidth * focalLength) / perWidth

# Initialize known parameters
KNOWN_DISTANCE = 24.0  # distance from camera to object in inches
KNOWN_WIDTH = 11.0     # width of the object in inches (e.g., width of standard A4 paper)

# Load the reference image that contains an object that has a known width
image = cv2.imread("/Users/namanmuktha/Desktop/cbithck/flower.jpeg")
if image is None:
    raise FileNotFoundError("Reference image not found. Please check the path.")
marker = find_marker(image)
focalLength = (marker[1][0] * KNOWN_DISTANCE) / KNOWN_WIDTH

# Load the image to find the size of new objects
image = cv2.imread("/Users/namanmuktha/Desktop/cbithck/flower.jpeg")
if image is None:
    raise FileNotFoundError("New image not found. Please check the path.")
marker = find_marker(image)
inches = distance_to_camera(KNOWN_WIDTH, focalLength, marker[1][0])

# Draw a bounding box around the image and display it
box = cv2.boxPoints(marker)
box = np.int0(box)
cv2.drawContours(image, [box], -1, (0, 255, 0), 2)
cv2.putText(image, f"{inches:.2f}in", (image.shape[1] - 200, image.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,
            2.0, (0, 255, 0), 3)
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


: 